In [10]:
import tensorflow as tf
import numpy as np

In [80]:
tf.reset_default_graph()

Construction Phase

In [81]:
n_inputs = 28 * 28
n_hidden1 = 500
n_hidden2 = 200
n_outputs = 10

In [82]:
X = tf.placeholder(name='X', shape=(None, n_inputs), dtype=tf.float32)
y = tf.placeholder(name='y', shape=(None), dtype=tf.int64)

In [29]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

Using custom neuron_layer() function (Run one of the following two cells)

In [30]:
with tf.name_scope('dnn1'):
    hidden1 = neuron_layer(X, n_hidden1, 'hidden1', tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, 'hidden2', tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, 'outputs')

Using TF library function

In [83]:
with tf.name_scope('dnn2'):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.elu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.elu)
    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

In [84]:
with tf.name_scope('loss'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(cross_entropy, name='loss')

In [85]:
learning_rate = 0.1
with tf.name_scope('train'):
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimiser.minimize(loss)

In [86]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [87]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Execution Phase

In [19]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [20]:
mnist.train.labels.shape[0]
mnist.train.num_examples // 50

1100

In [88]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size=batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_eval = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print('Epoch: {} Training Accuracy: {} Validation Accuracy: {}'.format(epoch, acc_train, acc_eval))
    saver_path = saver.save(sess, "/tmp/mnist_dnn_model_final.ckpt")

Epoch: 0 Training Accuracy: 0.9200000166893005 Validation Accuracy: 0.9405999779701233
Epoch: 1 Training Accuracy: 0.9800000190734863 Validation Accuracy: 0.9588000178337097
Epoch: 2 Training Accuracy: 0.9800000190734863 Validation Accuracy: 0.9710000157356262
Epoch: 3 Training Accuracy: 1.0 Validation Accuracy: 0.9728000164031982
Epoch: 4 Training Accuracy: 1.0 Validation Accuracy: 0.9765999913215637
Epoch: 5 Training Accuracy: 1.0 Validation Accuracy: 0.977400004863739
Epoch: 6 Training Accuracy: 1.0 Validation Accuracy: 0.9782000184059143
Epoch: 7 Training Accuracy: 1.0 Validation Accuracy: 0.9765999913215637
Epoch: 8 Training Accuracy: 1.0 Validation Accuracy: 0.9801999926567078
Epoch: 9 Training Accuracy: 1.0 Validation Accuracy: 0.9801999926567078
Epoch: 10 Training Accuracy: 1.0 Validation Accuracy: 0.9789999723434448
Epoch: 11 Training Accuracy: 1.0 Validation Accuracy: 0.9753999710083008
Epoch: 12 Training Accuracy: 1.0 Validation Accuracy: 0.9797999858856201
Epoch: 13 Trainin

In [ ]:
mnist.test.images[0, :].reshape((1, -1)).shape

In [89]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/mnist_dnn_model_final.ckpt")
    Z = logits.eval(feed_dict = {X: mnist.test.images})
    y_pred = np.argmax(Z, axis = 1)
    print(np.mean(y_pred == mnist.test.labels).astype(np.float32))

INFO:tensorflow:Restoring parameters from /tmp/mnist_dnn_model_final.ckpt
0.981
